In [24]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [25]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("classification_data/attack_img/attack_data.csv")
    shuffled_df = df.sample(len(df))

    print(shuffled_df.head)
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"classification_data/attack_img/{entry[0]}")

        prepared_y = np.delete(entry,0) * 3
        print(prepared_y)
        if prepared_y[0] >= 960:
            prepared_y[0] = prepared_y[0] - 960
        else:
            prepared_y[0] = -prepared_y[0]
            
        if prepared_y[1] >= 540:
            prepared_y[1] = prepared_y[1] - 540
        else:
            prepared_y[1] = -prepared_y[1]
        
        X.append(entry_img)
        y.append(prepared_y)

    
    X = np.asarray(X)
    y = np.asarray(y).astype(np.float32)
    return X, y


In [26]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

    return model

model = compile_model()

In [27]:
model.compile(optimizer='adam',
              loss='mean_squared_error')


In [28]:
X, y = create_input()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(y_train[0])

<bound method NDFrame.head of            filename    x    y
147  attack_147.jpg  384  107
316  attack_316.jpg  401  138
363  attack_363.jpg  178   54
207  attack_207.jpg  395  214
9      attack_9.jpg  281   52
..              ...  ...  ...
198  attack_198.jpg  378   63
159  attack_159.jpg  364  158
371  attack_371.jpg  293   38
237  attack_237.jpg  219  103
4      attack_4.jpg  402   99

[395 rows x 3 columns]>
[1152 321]
[1203 414]
[534 162]
[1185 642]
[843 156]
[174 156]
[1023 540]
[552 150]
[1197 336]
[1158 453]
[783 45]
[1125 291]
[1095 360]
[1311 54]
[726 744]
[1161 414]
[1155 198]
[1125 393]
[1050 117]
[1110 591]
[951 603]
[954 591]
[1056 606]
[1209 537]
[1158 252]
[984 552]
[1245 504]
[1056 324]
[1062 243]
[1011 234]
[1068 216]
[1158 462]
[1095 255]
[705 447]
[1428 783]
[1113 297]
[495 402]
[1218 414]
[972 279]
[1125 312]
[1050 261]
[1188 288]
[1194 246]
[1257 57]
[786 216]
[1119 177]
[1146 201]
[1725 351]
[1275 324]
[1164 414]
[1020 609]
[330 354]
[1107 195]
[1095 594]
[1341 51

In [29]:
model.fit(X_train, y_train, epochs=80, validation_data=(X_val, y_val))


Epoch 1/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: 3293240.7500 - val_loss: 240838.6406
Epoch 2/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 188229.0000 - val_loss: 72240.7266
Epoch 3/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 87706.7109 - val_loss: 69411.2578
Epoch 4/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 87884.2422 - val_loss: 72502.9531
Epoch 5/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 68290.8203 - val_loss: 89909.0000
Epoch 6/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 67115.5000 - val_loss: 72060.8125
Epoch 7/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 978ms/step - loss: 47958.3789 - val_loss: 87524.8359
Epoch 8/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 983ms/step - loss: 39011.6211 - val_loss: 80118.5312
Epoch 9/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 989ms/step - loss: 29989.7832 - val_loss: 81336.6016
Epoch 10/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 983ms/step - loss: 20672.7109 - val_loss: 85033.4922
Epoch 11/80
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 969ms/step - loss: 1

In [30]:
model.save('mouse_attack_fhd_normalized.h5')

In [32]:
model_rap = tf.keras.models.load_model('mouse_attack_fhd_normalized.h5')


for n in range(len(y_val)):
    x_real, y_real = int(y_val[n][0]), int(y_val[n][1])
    print(x_real, y_real)
    img = np.asarray(X_val[n])
    if x_real < 0:
        x_real = -x_real 
    else:
        x_real = x_real + 960
    
    if y_real < 0:
        y_real = -y_real
    else:
        y_real = y_real + 540
        
    
    tbp = np.expand_dims(img, axis=0)
    img = cv2.resize(img,(1920,1080))
    predictions = model_rap.predict(tbp)
    
    x, y = int(predictions[0][0]), int(predictions[0][1])
    if x < 0:
        x = -x 
    else:
        x = x + 960
    
    if y < 0:
        y = -y
    else:
        y = y + 540
        
    print(x, y, "\n",x_real, y_real)
    cv2.circle(img, (x,y), 10, (0,0,255), -1)
    cv2.circle(img, (x_real, y_real), 10, (255,0,0), -1)
    cv2.imshow('ss', img)
    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

51 -267
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1089 215 
 1011 267
126 -201
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1051 232 
 1086 201
120 -282
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
147 199 
 1080 282
-468 -258
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
982 120 
 468 258
174 -189
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1102 151 
 1134 189
186 -171
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
985 54 
 1146 171
234 -246
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1124 229 
 1194 246
153 -429
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
196 209 
 1113 429
105 -174
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1027 162 
 1065 174
186 -405
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1011 238 
 1146 405
198 -453
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
976 306 
 1158 453
-816 -108
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1412 105 
 816 108
321 -195
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
171 133 
 1281 195
-657 -309
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
194 221 
 657 309
258 -375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
132 85 
 1218 375
216 -

In [28]:
model_rap = tf.keras.models.load_model('my_mobilenet_model.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
model_rap.save('class_2.h5')

Epoch 1/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7854 - loss: 0.5956 - val_accuracy: 0.7400 - val_loss: 0.7340
Epoch 2/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.8160 - loss: 0.5927 - val_accuracy: 0.7000 - val_loss: 0.7350
Epoch 3/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7564 - loss: 0.6643 - val_accuracy: 0.6900 - val_loss: 0.7348
Epoch 4/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7663 - loss: 0.5880 - val_accuracy: 0.7200 - val_loss: 0.7174
Epoch 5/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7939 - loss: 0.5774 - val_accuracy: 0.7500 - val_loss: 0.6913
Epoch 6/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7758 - loss: 0.5911 - val_accuracy: 0.7700 - val_loss: 0.6796
Epoch 7/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8515 - loss: 0.5431 - val_accuracy: 0.7200 - val_loss: 0.6956
Epoch 8/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8156 - loss: 0.5335 - val_accuracy: 0.7600 - val_loss: